In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
import plotly
import plotly.express as px
import plotly.io as pio
pio.renderers.default = "iframe"
import plotly.graph_objects as go

In [35]:
#import dataset pulito
df=pd.read_csv('DATA/pre-processed/clean_app_data.csv')

## Qual'è la strategia di pricing delle app nel campione estratto dal Play Store?

In [36]:
fig = px.pie(names=['Gratis','Pagamento'], 
             values=df['Type'].value_counts().values,
             title='App Gratis versus a Pagamento',
             color=['Gratis','Pagamento'],
             color_discrete_map={"Gratis":"green",
                                 "Pagamento":"goldenrod"}
            )
fig.update_traces(textposition='inside', textinfo='percent')
fig.show()

**La maggior parte delle app sono gratis. Meno del 10% è a pagamento.**

&#x1F534;Le app gratuite potrebbero prevedere acquisti in app o offrire un upgrade per evitare pubblicità (c.d. modello freemium).

&#x1F534; Le app gratuite potrebbero essere temporaneamente gratuite, per poi ad esempio diventare a pagamento raggiunta la massa critica di utenti.

&#x1F534; Le app gratuite potrebbero essere mezzi a supporto di un business già esistente (versione mobile di un sito di e-commerce).


In [5]:
free = df[df.Type=='Free'].Category.value_counts().sort_index().to_frame()
paid = df[df.Type=='Paid'].Category.value_counts().sort_index().to_frame()
total = df.Category.value_counts().sort_index().to_frame()
free.columns=['free']
paid.columns=['paid']
total.columns=['total']
dist = free.join(paid).join(total)
dist.fillna(0,inplace=True)
dist ['paid_per'] = dist.paid*100/dist.total
dist ['free_per'] = dist.free*100/dist.total
tuple_free = tuple(dist.free_per.tolist())
tuple_paidapps = tuple(dist.paid_per.tolist())
cat = list(set(dist.index))

In [37]:
df_10_cat_per_paid = dist.sort_values(by='paid_per',ascending=False)
fig = go.Figure(data=[go.Table(
    header=dict(values=['CATEGORIE','N° di APPS', '% DI APPS A PAGAMENTO','% DI APPS GRATIS'],
                align='left',
                line_color='darkslategray',
                fill_color='royalblue',
                font=dict(color='white', size=12)),
    cells=dict(values=[df_10_cat_per_paid.index,df_10_cat_per_paid.total,np.round(df_10_cat_per_paid.paid_per.values,2),np.round(df_10_cat_per_paid.free_per.values,2)],
               align='left',
               line_color='darkslategray',
               fill_color='white',
               font = dict(color = 'black', size = 11)))
])

fig.update_layout(title='Categorie ordinate per % di APPs a pagamento')
fig.show()

## In media, qual' è il prezzo di un app a pagamento?

In [38]:
fig = px.strip(df[df.Type=='Paid'], y='Price',hover_data=['App'])
fig.update_layout(title='Distribuzione della variabile Prezzo')
fig.show()

In [39]:
print('Statistiche Descrittive della Variabile Prezzo')
df[df.Type=='Paid'].Price.describe().to_frame()

Statistiche Descrittive della Variabile Prezzo


,Price
count,731.000000
mean,4.846594
std,8.252596
min,0.990000
25%,1.490000
50%,2.990000
75%,4.990000
max,109.990000


In [40]:
fig = px.box(df[df.Type=='Paid'], y="Price",title="Box plot del prezzo",hover_data=['App','Category'])
fig.update_layout()
fig.show()

In [41]:
fig = px.box(df[df.Type=='Paid'], y="Price",title="Box plot del prezzo per categoria",color='Category')
fig.update_layout()
fig.show()

In [42]:
print('Statistiche Descrittive della Variabile Prezzo')
df[df.Type=='Paid'].Price.describe().to_frame()

Statistiche Descrittive della Variabile Prezzo


,Price
count,731.000000
mean,4.846594
std,8.252596
min,0.990000
25%,1.490000
50%,2.990000
75%,4.990000
max,109.990000


&#x1F534;La maggior parte delle app ha un prezzo minore di 5 USD

&#x1F534;Le app più costose si concentrano nelle categorie *Medical*, *Finance* e *Business*

&#x1F534;La categoria Medical, oltre avere app in media più costose e anche una delle categorie con la più alta percentuale di app a pagamento

### Redditività di un app

In [43]:
#creazione di una nuova variabile
df.Price_X_Installs=np.nan
for i in df[df.Type=='Paid'].index:
    df.loc[i,'Price_X_Installs']=df.loc[i,'Price']*df.loc[i,'Installs']

In [44]:
print(f'Il {round(len(df[df["Price_X_Installs"]>0])/len(df[df["Type"]=="Paid"]),2)} % delle app a pagamento ha creato un maggiore di 0...')

Il 1.0 % delle app a pagamento ha creato un maggiore di 0...


In [45]:
fig = px.box(df[(df.Type=='Paid')&(df.Installs>0)], y='Price_X_Installs',hover_data=['App','Category'])
fig.update_layout(title='Distribuzione della variabile InstallsxPrice')
fig.show()

In [46]:
fig = px.box(df[(df.Type=='Paid')&(df.Installs>0)], y="Price_X_Installs",title="Box plot del InstallsxPrice per categoria",color='Category',hover_data=['App','Category'])
fig.update_layout()
fig.show()

### Rapporto tra prezzo e rating di un app

In [47]:
df_paid = df[df['Type']=='Paid']
fig=px.scatter( df_paid, "Price", "Rating")
fig.show()

In [49]:
df_paid[['Price','Rating']].corr()

,Price,Rating
Price,1.00000,-0.02882
Rating,-0.02882,1.00000


### Conclusioni

&#x1F534; Ci sono sicuramente altre leve più importanti che determinano la redditività di un app che il prezzo di download, soprattutto nel lungo periodo (in-app purchases,pubblicità,offerta di servizi premium a pagamento).

&#x1F534; Il dataset può essere comunque utile per decidere come prezzare un'app quando viene lanciata oppure come calibrare nel tempo il prezzo di un'app già esistente.

&#x1F534; Non sembra esserci una la relazione tra il rating dell'app e il prezzo.